In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("PreprocessOps") \
.config("spark.executor.memory","3g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [2]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("simpledataset.csv")

In [3]:
df.toPandas().head()

,sirano,isim,yas,cinsiyet,meslek,sehir,aylik_gelir,mal_mulk
0,1,Cemal,35,E,Isci,Ankara,3500.0,araba
1,2,ceyda,42,K,Memur,Kayseri,4200.0,araba|ev
2,3,Timur,30,None,Müzüsyen,Istanbul,9000.0,araba|ev|yazlık
3,4,Burcu,29,K,Pazarlamacı,Ankara,4200.0,araba
4,5,Yasemin,23,K,Pazarlamaci,Bursa,4800.0,araba


# VERİ SETİNE ETİKET EKLEME


// Sınıflandırma hedef değişken (etiket-label) yaratmak adına

// Geliri 7000 üstü olanların ekonomik_durumu iyi diyelim.

In [4]:
from pyspark.sql.functions import *

In [5]:
df1 = df.withColumn("ekonomik_durum", 
when(col("aylik_gelir") > 7000, "iyi").otherwise("kötü")
         
)

df1.toPandas().head()

,sirano,isim,yas,cinsiyet,meslek,sehir,aylik_gelir,mal_mulk,ekonomik_durum
0,1,Cemal,35,E,Isci,Ankara,3500.0,araba,kötü
1,2,ceyda,42,K,Memur,Kayseri,4200.0,araba|ev,kötü
2,3,Timur,30,None,Müzüsyen,Istanbul,9000.0,araba|ev|yazlık,iyi
3,4,Burcu,29,K,Pazarlamacı,Ankara,4200.0,araba,kötü
4,5,Yasemin,23,K,Pazarlamaci,Bursa,4800.0,araba,kötü


# 1. StringIndexer AŞAMASI

In [7]:
!pip install pyspark


In [8]:

from pyspark.ml.feature import IndexToString, StringIndexer, OneHotEncoder, VectorIndexer, Interaction, VectorAssembler, Normalizer, StandardScaler, RobustScaler, MinMaxScaler

In [9]:
meslek_indexer = StringIndexer() \
.setInputCol("meslek") \
.setOutputCol("meslek_index")

In [10]:
meslek_indexer_model = meslek_indexer.fit(df1)
meslek_indexer_df = meslek_indexer_model.transform(df1)

In [11]:
meslek_indexer_df.toPandas().head(15)

,sirano,isim,yas,cinsiyet,meslek,sehir,aylik_gelir,mal_mulk,ekonomik_durum,meslek_index
0,1,Cemal,35,E,Isci,Ankara,3500.0,araba,kötü,5.0
1,2,ceyda,42,K,Memur,Kayseri,4200.0,araba|ev,kötü,1.0
2,3,Timur,30,None,Müzüsyen,Istanbul,9000.0,araba|ev|yazlık,iyi,2.0
3,4,Burcu,29,K,Pazarlamacı,Ankara,4200.0,araba,kötü,7.0
4,5,Yasemin,23,K,Pazarlamaci,Bursa,4800.0,araba,kötü,3.0
5,6,Ali,33,E,Memur,Ankara,4250.0,ev,kötü,1.0
6,7,Dilek,29,K,Pazarlamaci,Istanbul,7300.0,araba|yazlık,iyi,3.0
7,8,Murat,31,E,Müzüsyen,Istanbul,12000.0,araba|ev|dükkan|yazlık,iyi,2.0
8,9,Ahmet,33,E,Doktor,Ankara,180000.0,araba|ev|yazlık,iyi,0.0
9,10,Muhittin,46,E,Berber,Istanbul,12000.0,araba|ev|dükkan,iyi,4.0


In [12]:
df1.groupBy(col("meslek")) \
.agg(count("*").alias("sayi")) \
.sort(desc("sayi")) \
.toPandas().head(10)

,meslek,sayi
0,Memur,2
1,Müzüsyen,2
2,Pazarlamaci,2
3,Doktor,2
4,Isci,1
5,Tuhafiyeci,1
6,Müzisyen,1
7,Pazarlamacı,1
8,Berber,1
9,Tornacı,1


In [13]:
sehir_indexer = StringIndexer() \
.setInputCol("sehir") \
.setOutputCol("sehir_index")

In [14]:
sehir_indexer_model = sehir_indexer.fit(meslek_indexer_df)
sehir_indexer_df = sehir_indexer_model.transform(meslek_indexer_df)

In [15]:
sehir_indexer_df.toPandas().head(15)

Py4JJavaError: An error occurred while calling o166.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 413, LAPTOP-349QG0AA, executor driver): org.apache.spark.SparkException: Failed to execute user defined function(StringIndexerModel$$Lambda$2861/0x00000001012a0840: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:729)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:340)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:872)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:872)
	at

# 2. OneHotEncoderEstimator AŞAMASI

In [18]:
encoder = OneHotEncoderEstimator() \
.setInputCols(["meslek_index","sehir_index"]) \
.setOutputCols(["meslek_encoded","sehir_encoded"])

In [19]:
encoder_model = encoder.fit(sehir_indexer_df)
encoder_df = encoder_model.transform(sehir_indexer_df)

In [20]:
encoder_df.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded
0,1,Cemal,35,Isci,Ankara,3500,kötü,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,5.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0)"
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,1.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,1.0,2.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)"


# 3. VectorAssembler AŞAMASI

In [23]:
assembler = VectorAssembler() \
.setInputCols(["yas","aylik_gelir","meslek_encoded","sehir_encoded"]) \
.setOutputCol("vectorized_features")

In [25]:
assembler_df = assembler.transform(encoder_df)

In [27]:
# Pandas dataframe head() metodundan satır truncate'i kaldırmak için 
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [29]:
assembler_df.select("vectorized_features").toPandas().head()

,vectorized_features
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
2,"(30.0, 9000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
3,"(29.0, 4200.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
4,"(23.0, 4800.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)"


# 4. LabelIndexer AŞAMASI

In [30]:
label_indexer = StringIndexer() \
.setInputCol("ekonomik_durum") \
.setOutputCol("label")

In [31]:
label_indexer_model = label_indexer.fit(assembler_df)
label_indexer_df = label_indexer_model.transform(assembler_df)

In [38]:
label_indexer_df.select("vectorized_features","label").toPandas().head(15)

,vectorized_features,label
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0
2,"(30.0, 9000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
3,"(29.0, 4200.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
4,"(23.0, 4800.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0
5,"(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
6,"(29.0, 7300.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
7,"(31.0, 12000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
8,"(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",1.0
9,"(46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0


# 5. StandardScaler AŞAMASI

In [40]:
math.sqrt(math.pow((35-33),2) + math.pow((18000-3500), 2))

14500.000137931034

In [41]:
math.sqrt(math.pow((35-33),2))

2.0

In [42]:
math.sqrt(math.pow((18000-3500), 2))

14500.0

In [44]:
scaler = StandardScaler() \
.setInputCol("vectorized_features") \
.setOutputCol("features")

In [45]:
scaler_model = scaler.fit(label_indexer_df)

In [47]:
scaler_df = scaler_model.transform(label_indexer_df)

In [48]:
scaler_df.select("features").toPandas().head(15)

,features
0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
1,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
2,"(4.292812263480104, 1.9859783574511787, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
3,"(4.149718521364101, 0.9267899001438834, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
4,"(3.29115606866808, 1.0591884573072952, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)"
5,"(4.722093489828115, 0.9378231132408343, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
6,"(4.149718521364101, 1.610849112154845, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
7,"(4.435906005596108, 2.6479711432682382, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
8,"(4.722093489828115, 3.9719567149023574, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
9,"(6.58231213733616, 2.6479711432682382, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"


DATACAMP ORNEK MİN MAX SCALE

In [ ]:
def min_max_scaler(df, cols_to_scale):
  # Takes a dataframe and list of columns to minmax scale. Returns a dataframe.
  for col in cols_to_scale:
    # Define min and max values and collect them
    max_days = df.agg({col: 'max'}).collect()[0][0]
    min_days = df.agg({col: 'min'}).collect()[0][0]
    new_column_name = 'scaled_' + col
    # Create a new column based off the scaled data
    df = df.withColumn(new_column_name, 
                      (df[col] - min_days) / (max_days - min_days))
  return df
  
df = min_max_scaler(df, cols_to_scale)
# Show that our data is now between 0 and 1
df[['DAYSONMARKET', 'scaled_DAYSONMARKET']].show()

# 6. Train-Test Ayırma AŞAMASI


In [49]:
train_df, test_df = scaler_df.randomSplit([0.8, 0.2], seed=142)

In [51]:
train_df.toPandas().head(15)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features,label,features
0,1,Cemal,35,Isci,Ankara,3500,kötü,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,5.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0)","(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
2,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,1.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(29.0, 4200.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(4.149718521364101, 0.9267899001438834, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
3,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,1.0,2.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)","(23.0, 4800.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0,"(3.29115606866808, 1.0591884573072952, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)"
4,7,Dilek,29,Pazarlamaci,Istanbul,7300,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(29.0, 7300.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.149718521364101, 1.610849112154845, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
5,8,Murat,31,Müzisyen,Istanbul,12000,iyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(31.0, 12000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.435906005596108, 2.6479711432682382, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
6,11,Hicaziye,47,Tuhafiyeci,Ankara,4800,kötü,5.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(47.0, 4800.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(6.725405879452164, 1.0591884573072952, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
7,12,Harun,43,Tornacı,Ankara,4200,kötü,4.0,0.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(43.0, 4200.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(6.15303091098815, 0.9267899001438834, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
8,13,Hakkı,33,Memur,Çorum,3750,kötü,0.0,3.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","(33.0, 3750.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",0.0,"(4.722093489828115, 0.8274909822713244, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)"
9,14,Gülizar,37,Doktor,İzmir,14250,iyi,3.0,4.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 1.0)","(37.0, 14250.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)",1.0,"(5.294468458292129, 3.1444657326310326, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175)"


In [52]:
test_df.toPandas().head(10)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features,label,features
0,3,Timur,30,Müzisyen,Istanbul,9000,iyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(30.0, 9000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.292812263480104, 1.9859783574511787, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
1,6,Ali,33,Memur,Ankara,4250,kötü,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(4.722093489828115, 0.9378231132408343, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
2,9,Ahmet,33,Doktor,Ankara,18000,iyi,3.0,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",1.0,"(4.722093489828115, 3.9719567149023574, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
3,10,Muhittin,46,Berber,Istanbul,12000,iyi,7.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(6.58231213733616, 2.6479711432682382, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
